Link dos Datasets: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews

In [1]:
import pandas as pd
import numpy as np
import chardet
import seaborn as sns
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [2]:
df_books = pd.read_csv('./datasets/books_data.csv')
df_books.head()

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN


In [ ]:
df_books = df_books[[ 'Title', 'authors', 'image', 'categories']].copy()
df_books.rename(columns={
    'Title': 'title'
}, inplace=True)
df_books

In [ ]:
df_ratings = pd.read_csv('./datasets/Books_rating.csv')
df_ratings.head()

In [ ]:
df_ratings = df_ratings[[ 'Title', 'User_id', 'review/score']].copy()
df_ratings.rename(columns={
    'Title': 'title',
    'User_id' : 'user_id',
    'review/score' : 'rating'
    
}, inplace=True)
df_ratings.dropna(inplace=True)
df_ratings


In [ ]:
df_ratings.describe()

In [ ]:
print(len(df_ratings['user_id'].unique()))

In [ ]:
user_rating_count = df_ratings.groupby('user_id').size().sort_values(ascending=False)
user_rating_count

In [ ]:
book_rating_count = df_ratings.groupby('title').size().sort_values(ascending=False)
book_rating_count

In [ ]:
while True:
    book_rating_count = df_ratings['title'].value_counts()
    filtered_books = book_rating_count[book_rating_count > 20].index
    user_rating_count = df_ratings[df_ratings['title'].isin(filtered_books)]['user_id'].value_counts()
    filtered_users = user_rating_count[user_rating_count > 20].index
    df_ratings_filtered = df_ratings[
        df_ratings['title'].isin(filtered_books) &
        df_ratings['user_id'].isin(filtered_users)
    ]
    if len(df_ratings_filtered) == len(df_ratings):
        break


    df_ratings = df_ratings_filtered.copy()
df_ratings_filtered.reset_index(drop=True, inplace=True)
print(df_ratings_filtered)

In [ ]:
user_rating_count = df_ratings.groupby('user_id').size().sort_values(ascending=False)
user_rating_count

In [ ]:
book_rating_count = df_ratings.groupby('title').size().sort_values(ascending=False)
book_rating_count

In [ ]:
sns.histplot(df_ratings['rating'], bins = 5)

In [ ]:
book_index = {book: idx for idx, book in enumerate(df_ratings['title'].unique())}
book_index.get('Pride and Prejudice')

In [ ]:
user_index = {user: idx for idx, user in enumerate(df_ratings['user_id'].unique())}
user_index.get('A30TK6U7DNS82R')

In [ ]:
df_ratings.loc[:, 'book_index'] = df_ratings['title'].map(book_index)
df_ratings.loc[:, 'user_index'] = df_ratings['user_id'].map(user_index)

df_ratings.to_csv('/kaggle/working/processed_ratings.csv', index=False)

# save books detail
filtered_books = df_books[df_books['title'].isin(df_ratings['title'].unique())]
filtered_books = filtered_books.merge(
    df_ratings[['title', 'book_index']].drop_duplicates(), 
    on='title',
    how='left'
)
filtered_books.to_csv('/kaggle/working/books.csv', index=False)


print(df_ratings)

In [ ]:
book_pivot = df_ratings.pivot_table(columns='user_index', index='book_index', values="rating")
book_pivot.head(100)

In [ ]:
book_pivot.fillna(0,inplace=True)
book_pivot.head()

In [ ]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot)

In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')
model.fit(book_sparse)

In [ ]:
distances, suggestions = model.kneighbors(book_pivot.iloc[237, :].values.reshape(1, -1), n_neighbors=6)
distances

In [ ]:
suggestions

In [ ]:
for i in range(len(suggestions)):
  print(book_pivot.index[suggestions[i]])

In [ ]:
book_pivot.index[3]